In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functools
import inspect
import types

In [3]:
def foo(a, b=3, *args, c=4, **kwargs):
    return a, b, args, c, kwargs

In [5]:
foo(1, 2, 3)

(1, 2, (3,), 4, {})

In [6]:
def copy_func(func):
    new_func = types.FunctionType(func.__code__, func.__globals__, 
                                  func.__name__, func.__defaults__,
                                  func.__closure__)
    new_func.__kwdefaults__ = func.__kwdefaults__.copy()
    return functools.update_wrapper(new_func, func)

In [11]:
def swap_params(func, old2new):
    new_func = copy_func(func)
    sig = inspect.signature(new_func)
    kw_defaults = func.__kwdefaults__
    names, params = map(list, zip(*sig.parameters.items()))
    for old, new in old2new.items():
        idx = names.index(old)
        default = kw_defaults.get(old)
        kwargs = {'default': default} if default else {}
        params[idx] = inspect.Parameter(new, params[idx].kind, **kwargs)
    new_func.__signature__ = sig.replace(parameters=params)
    return new_func

In [8]:
res = swap_params(foo, {'b': 'inserted', 'a': 'removed', 'c': 'command'})
res

<function __main__.foo(removed, inserted, *args, command=4, **kwargs)>

In [9]:
res(8, 9, 10)

(8, 9, (10,), 4, {})

In [10]:
foo(8, 9, 10)

(8, 9, (10,), 4, {})